In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '..')

from sim.LoadBalancer import LoadBalancer
from sim.Source import AutoRegressiveArrivalSchedule, Source, ExponentialSource
from sim.Environment import Environment
from sim.GELoadBalancer import GreedyEpsilonLoadBalancer
import numpy as np
import random
import matplotlib.pyplot as plt
import math
from optServers import getOptServer


## Arrival process
use number of arrivals and number of cancelled to define autoregressive process

In [ ]:
def sigmoid(x):
      return 1 / (1 + np.exp(-x))

In [ ]:
X0 = np.array([18000, 0])
X = X0
A = np.array([1,0.05])
arrivalProcess = lambda X: X.T @ A 

arr = []
canc = []
for i in range(0,100):
    arrs  = arrivalProcess(X) + random.normalvariate(0,100)
    arr.append(arrs)
    cancelled = max(arrs-18000,0)
    canc.append(cancelled)
    X = np.array([arrs, cancelled])

plt.plot(arr)
plt.figure()
plt.plot(canc)


### Test the process

In [ ]:
def sigmoid(x):
      return 1 / (1 + np.exp(-x))

In [ ]:
#from sklearn.linear_model import LinearRegression
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

stopTime = 4*7*24*60*60
env = Environment(stopTime=stopTime)

#model = LinearRegression()
#model = DecisionTreeRegressor()
#model  = MLPRegressor(hidden_layer_sizes=[10], activation='tanh')
model  = RandomForestRegressor(n_estimators=10)
eta = lambda t: sigmoid(2-0.01*t)
#eta = 0.4
#sigmoid(2/(0.1*t+2)-0.5)
periodLength = 1*60*60 #half an hour per period -> schedule repeated two times in 12 hours
loadBalancer = GreedyEpsilonLoadBalancer(nServers=10, environment=env, model = model, eta=eta, nServerRange=[10,30], usePartialFit=False, periodLength=periodLength)

requestTypes = [(0.5,1,0.1,10), (0.5,2,0.2,10)] #(prob, mu, sigma, cancelTime)
arrivalsPerSecond = 10 #starting value
source = ExponentialSource(arrivalsPerSecond, requestTypes, loadBalancer, env)

schedule = np.array([arrivalsPerSecond]) #12 periods
arrivalSchedule = AutoRegressiveArrivalSchedule(periodLength,arrivalSchedule=schedule, environment=env, loadBalancer=loadBalancer, source=source, maxArrivals=20, A=[0,1,0,0,0,0.3,0])
env.run(debug=False)

In [ ]:
import pandas as pd
pd.DataFrame(list(loadBalancer.X)).head(10)

In [ ]:
print(loadBalancer.X.shape, loadBalancer.y.shape)


In [ ]:
X = loadBalancer.X[-1,:]
X
pred = []
for n  in range(0,40):
    X_ = np.append(X[:-1], n)
    rewardHat = loadBalancer.model.predict(X_[None,:])[0]
    pred.append(rewardHat)

plt.plot(range(0,40), pred)

In [ ]:
plt.scatter(env.log['numberOfServers'][::2], env.log['reward'])

In [ ]:
plt.plot(env.log['numberOfServers'])


In [ ]:
plt.plot(env.log['arrivalsPerSecond'])
plt.ylabel('Arrivals')
plt.xlabel('Time (seconds)')



In [ ]:
pred = np.array(env.log['numberOfServers'][::2])
real = np.array([getOptServer(_) for _ in env.log['arrivalsPerSecond'][1:-1]])
ged = np.array(env.log['greedyEpsilonActionType'])
print(len(pred), len(real))
plt.figure(figsize=(15,5))
plt.plot(pred, label='Chosen #servers')
plt.plot(real, color='red', label='Optimal #servers')
plt.legend()
plt.xlabel('Time (seconds)')
plt.ylabel('#servers')
plt.figure()
plt.scatter(pred[ged==1],real[ged==1])

In [ ]:
pred = np.array(env.log['numberOfServers'][::2])
real = np.array([getOptServer(_) for _ in env.log['arrivalsPerSecond'][1:-1]])
ged = np.array(env.log['greedyEpsilonActionType'])
print(len(pred), len(real))
plt.figure(figsize=(15,5))
plt.plot(pred[ged==1], label='Chosen #servers')
plt.plot(real[ged==1], color='red', label='Optimal #servers')
plt.legend()
plt.xlabel('Time (seconds)')
plt.ylabel('#servers')
plt.figure()
plt.scatter(pred[ged==1],real[ged==1])

In [ ]:
plt.plot(env.log['eta'])
plt.ylabel('$\eta$')
plt.xlabel('Period')

In [ ]:
plt.plot(env.log['eta'])
plt.figure()
plt.plot(env.log['numberOfServers'])
plt.figure()
plt.plot(env.log['greedyEpsilonActionType'])
plt.figure()
plt.plot(env.log['reward'])
plt.figure()
plt.hist(env.logTime['arrivalEvent'], bins=int(stopTime/60/60));
plt.figure()
plt.hist(env.logTime['requestProcessed'], bins=int(stopTime/60/60));
plt.figure()
plt.hist(env.logTime['requestCancelled'], bins=int(stopTime/60/60));

In [ ]:
plt.plot(np.array(env.log['reward'])[np.array(env.log['greedyEpsilonActionType'])==1.0])
plt.plot(np.array(env.log['numberOfServers'])[np.array(env.log['greedyEpsilonActionType'])==1.0])

In [ ]:
plt.plot(np.array(env.log['numberOfServers'][::2])[np.array(env.log['greedyEpsilonActionType'])==1.0])

In [ ]:
plt.plot(env.log['arrivalsPerSecond'])

In [ ]:
plt.scatter(loadBalancer.X[:-1,2],loadBalancer.y)
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
plt.scatter(env.log['numberOfServers'][0::2],env.log['reward'])

In [ ]:

def sigmoid(x):
      return 1 / (1 + np.exp(-x))

x = np.arange(0,120,1)
plt.plot(x, sigmoid(2/(0.1*x+2)-0.5))
plt.plot(x, sigmoid(1-0.1*x))

In [ ]:
x = np.arange(0,1000,1)
plt.plot(x, sigmoid(2-0.01*x))